# Import Requirements

In [1]:
import tensorflow as tf
import glob
import csv
import time
import os
import random

In [2]:
#Major defines
N_INPUTS  = 32*32*3
N_CLASSES = 10
BATCH_SIZE = 64
data_path_train = 'train_old/'
data_path_labels = 'labels_old.txt'
data_path_test = 'test_old/'

In [3]:
def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

# Generating training data
filenames = [data_path_train+file for file in os.listdir(data_path_train)]  #get all filenames
random.shuffle(filenames)
labels = [file.split('_')[-1].split('.')[0] for file in filenames ] #get label string name eg. frog, ship...
#labels = list(map(int,labels))

id2label, label2id = get_label_mapping(data_path_labels) 
labels = [label2id[label] for label in labels if label in label2id] #map label  name to id number eg  ship -> 8

train_size = int(((len(filenames)*0.95)//BATCH_SIZE)*BATCH_SIZE)
val_size = int(((len(filenames)*0.05)//BATCH_SIZE)*BATCH_SIZE)

In [4]:
tf.reset_default_graph()

In [5]:
# Create input dataset and generate batches of data
def _parse_function(filenames, labels):
#     one_hot = tf.one_hot(labels, N_CLASSES)   #because using sparse cross entropy
    
    img_string = tf.read_file(filenames)
    img_decoded = tf.image.decode_png(img_string, channels=3)
    
    #Reserved for image augmentation
    float_image = tf.image.per_image_standardization(img_decoded)
    
    
#     float_image = tf.image.random_flip_left_right(float_image, seed = 1)
#     float_image = tf.contrib.image.rotate(float_image, np.random.randint(-15,16)) 
#     float_image = tf.image.random_brightness(float_image, 0.3)
#     float_image = tf.image.random_contrast(float_image, 0.1, 0.9)
#     float_image = tf.image.random_saturation(float_image, 0.1, 0.9)
    
    img_decoded = tf.reshape(float_image , [-1])  #flatten
    return img_decoded, labels

# Train data
data = tf.data.Dataset.from_tensor_slices((filenames, labels))
data = data.map(_parse_function, num_parallel_calls = 3)
data = data.batch(BATCH_SIZE)
data = data.shuffle(buffer_size=2000)    #random number 
iterator = data.make_initializable_iterator()
train_batch = iterator.get_next()  #next_element is tensor of (img_train, y_train)

# Validation data
if(len(filenames[train_size:(train_size+val_size)])>0):
    val = tf.data.Dataset.from_tensor_slices((filenames[train_size:(train_size+val_size)], labels[train_size:(train_size+val_size)]))   #validation batchval = tf.data.Dataset.from_tensor_slices((filenames[train_size:(train_size+val_size)], labels[train_size:(train_size+val_size)]))   #validation batch
else:
    val = tf.data.Dataset.from_tensor_slices((filenames[-1*BATCH_SIZE:], labels[-1*BATCH_SIZE:]))
val = val.map(_parse_function, num_parallel_calls = 3)
val = val.batch(5000)
val_iterator = val.make_initializable_iterator()
validation_batch = val_iterator.get_next()



# For test images
def _parse_function_test(filenames):
    img_string = tf.read_file(filenames)
    img_decoded = tf.image.decode_png(img_string, channels=3)
    
    #Reserved for image augmentation
    float_image = tf.image.per_image_standardization(img_decoded)
    img_decoded = tf.reshape(float_image , [-1])  #flatten
    
    return img_decoded

filenames_test = [data_path_test+file_t for file_t in os.listdir(data_path_test)]
filenames_test.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))


#For test data
data_test = tf.data.Dataset.from_tensor_slices(filenames_test)
data_test = data_test.map(_parse_function_test)
data_test = data_test.batch(2000)
test_iterator = data_test.make_initializable_iterator()
X_t = test_iterator.get_next() 

In [6]:
def conv_model(X, N_CLASSES, reuse, is_training):
    
    with tf.variable_scope('Conv', reuse = reuse): #to reuse weights and biases for testing
        
        input_layer = tf.reshape(X, [-1, 32, 32, 3])

        conv1_1 = tf.layers.conv2d(
          inputs=input_layer,
          filters=64,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv1_2 = tf.layers.conv2d(
          inputs=conv1_1,
          filters=64,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu) 
       
        pool1 = tf.layers.max_pooling2d(
            inputs = conv1_2,
            pool_size = 2,
            strides = 2,
            padding = "same",
            data_format='channels_last')
        
        conv2_1 = tf.layers.conv2d(
          inputs=pool1,
          filters=128,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv2_2 = tf.layers.conv2d(
          inputs=conv2_1,
          filters=128,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu) 
       
        pool2 = tf.layers.max_pooling2d(
            inputs = conv2_2,
            pool_size = 2,
            strides = 2,
            padding = "same",
            data_format='channels_last')
        
        conv3_1 = tf.layers.conv2d(
          inputs=pool2,
          filters=256,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv3_2 = tf.layers.conv2d(
          inputs=conv3_1,
          filters=256,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv3_3 = tf.layers.conv2d(
          inputs=conv3_2,
          filters=256,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu) 
       
        pool3 = tf.layers.max_pooling2d(
            inputs = conv3_3,
            pool_size = 2,
            strides = 2,
            padding = "same",
            data_format='channels_last')
        
        conv4_1 = tf.layers.conv2d(
          inputs=pool3,
          filters=512,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv4_2 = tf.layers.conv2d(
          inputs=conv4_1,
          filters=512,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv4_3 = tf.layers.conv2d(
          inputs=conv4_2,
          filters=512,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu) 
       
        pool4 = tf.layers.max_pooling2d(
            inputs = conv4_3,
            pool_size = 2,
            strides = 2,
            padding = "same",
            data_format='channels_last')
        
        conv5_1 = tf.layers.conv2d(
          inputs=pool4,
          filters=512,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv5_2 = tf.layers.conv2d(
          inputs=conv5_1,
          filters=512,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        conv5_3 = tf.layers.conv2d(
          inputs=conv5_2,
          filters=512,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu) 
       
        pool5 = tf.layers.max_pooling2d(
            inputs = conv5_3,
            pool_size = 2,
            strides = 2,
            padding = "same",
            data_format='channels_last')
        
        flatten = tf.layers.flatten(pool5)

        dense1 = tf.layers.dense(
            inputs = flatten,
            units = 1024,
            activation= tf.nn.relu)

#         dropout2 = tf.layers.dropout(
#             inputs = dense1,
#             rate=0.25,
#             training = is_training)

        dense2 = tf.layers.dense(
            inputs = dense1,
            units = 1024,
            activation= tf.nn.relu)
        
        dense3 = tf.layers.dense(
            inputs = dense2,
            units = N_CLASSES)
        
        if is_training:
            last_layer = dense3     #using sparse cross entropy so no need to apply softmax here
        else:
            last_layer = tf.nn.softmax(dense3)   #for inference

        #FOR FUTURE
    #     predictions = {
    #       # Generate predictions (for PREDICT and EVAL mode)
    #       "classes": tf.argmax(input=logits, axis=1),
    #       # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
    #       # `logging_hook`.
    #       "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    #       }

    #     if mode == tf.estimator.ModeKeys.PREDICT:
    #         return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

        return last_layer

In [7]:
X,y = train_batch

valX, valY = validation_batch

global_step = tf.Variable(0, dtype=tf.int32, trainable = False, name='global_step')

train_output = conv_model(X, N_CLASSES, reuse = False, is_training = True)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits (labels = y, logits = train_output))
learning_rate = tf.placeholder(tf.float32)
optimizer = tf.train.AdamOptimizer().minimize(cost, global_step = global_step)

#NOTE: THIS IS VERY INEFFICIENT. IMPROVE THIS BY USING feed_dict

# Evaluate model with train data
test_output = conv_model(X, N_CLASSES, reuse=True, is_training=False)
correct_pred = tf.equal(tf.argmax(test_output, 1, output_type=tf.int32), y)
train_accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# # Evaluate model with Validation data
val_test_output = conv_model(valX, N_CLASSES, reuse=True, is_training=False)
val_pred = tf.equal(tf.argmax(val_test_output, 1, output_type=tf.int32), valY)
val_accuracy = tf.reduce_mean(tf.cast(val_pred, tf.float32))

#for test data
test_r = conv_model(X_t, N_CLASSES, reuse=True, is_training=False)
test_pred = tf.argmax(test_r, 1 , output_type=tf.int32)

In [8]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
N_EPOCHS = 5
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)   #initialize variables
    sess.run(iterator.initializer)
    
    
    epochCount = 1
        
    while (epochCount < N_EPOCHS):
        startTime = time.time()
        while True:
            try:
                sess.run(optimizer)
                
            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
                tr_loss, tr_acc = sess.run([cost, train_accuracy])
                
                sess.run(val_iterator.initializer)
                val_acc = sess.run(val_accuracy)
                print("Epoch {}    Loss: {:,.4f}    Train Accuracy: {:,.2f}    Val Accuracy: {:,.2f}    Time: {:,.2f}"         
                      .format(epochCount, tr_loss, tr_acc, val_acc, time.time() - startTime))
                
               
                epochCount += 1
                break
                
            except KeyboardInterrupt:
                print ('\nTraining Interrupted at epoch %d' % epochCount)
            
                epochCount = N_EPOCHS + 1
                break
    print ('Done Training')
    #Save the model
    save_path = saver.save(sess, 'checkpoints/GLRC', global_step = global_step )
    print ('Model saved at %s' % save_path)
    
    sess.run(test_iterator.initializer)
    test_predictions = []
    while True:
        try:
            test_predictions.extend(sess.run(test_pred))
            
        except tf.errors.OutOfRangeError:
            break
            
        except KeyboardInterrupt:
            print ("Prediction interrupted")
            break
            
        except:
            print ()
            break
            
    predictions_csv = open('GLRC_'+str(epochCount)+'.csv', 'w')
    header = ['id','label']
    with predictions_csv:
        writer = csv.writer(predictions_csv)
        writer.writerow((header[0], header[1]))
        for count, row in enumerate(range(len(test_predictions))):
            writer.writerow((count, test_predictions[count]))
        print("Writing complete")

Epoch 1    Loss: 2.3046    Train Accuracy: 0.06    Val Accuracy: 0.11    Time: 96.27
Epoch 2    Loss: 2.3039    Train Accuracy: 0.11    Val Accuracy: 0.10    Time: 70.61
Epoch 3    Loss: 2.3022    Train Accuracy: 0.08    Val Accuracy: 0.10    Time: 70.55
Epoch 4    Loss: 2.3008    Train Accuracy: 0.14    Val Accuracy: 0.10    Time: 70.68
Done Training
Model saved at checkpoints/GLRC-2813
Writing complete
